In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
from PIL import Image
import numpy as np
from tqdm import tqdm
import torchvision.transforms.functional as f
import torch
import h5py

In [ ]:
def inverse_normalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

In [ ]:
patches = list(Path("/lustre/groups/shared/histology_data/TCGA/ACC/patches").glob("**/*.h5"))
# patches = list(Path("/lustre/groups/shared/tcga/CRC/patches/512px_crc_wonorm_complete_diag_frozen").glob("**/*.jpeg"))
# patches = np.loadtxt("/lustre/groups/shared/histology_data/TCGA/CRC/patches/512px_crc_wonorm_complete_diag_frozen.txt", dtype=str, max_rows=100).tolist()

In [ ]:
from dinov2.data import DataAugmentationDINO

In [ ]:
data_transform = DataAugmentationDINO(
    (1., 1.), #cfg.crops.global_crops_scale,
    (0.32, 0.32), #cfg.crops.local_crops_scale,
    8, #cfg.crops.local_crops_number,
    224, #global_crops_size=cfg.crops.global_crops_size,
    local_crops_size=98, #cfg.crops.local_crops_size,
)

In [ ]:
patch = Image.open(patches[70]).convert(mode="RGB")

In [ ]:
out = data_transform(patch)

In [ ]:
rev = inverse_normalize(tensor=out['global_crops'][1], mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
img = f.to_pil_image(rev)
img

In [ ]:
rev = inverse_normalize(tensor=out['local_crops'][1], mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
img = f.to_pil_image(rev)
img

In [ ]:
patch

In [ ]:
# random crop to (256, 256)
h, w = patch.size
i = torch.randint(0, h - 224 + 1, size=(1,)).item()
j = torch.randint(0, w - 224 + 1, size=(1,)).item()
patch = f.crop(patch, i, j, 224, 224)

out = data_transform(patch)
rev = inverse_normalize(tensor=out['global_crops'][1], mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
img = f.to_pil_image(rev)
img

In [ ]:
# rev = inverse_normalize(tensor=out['global_crops_teacher'][1], mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
# img = f.to_pil_image(rev)
img = f.to_pil_image(out['global_crops_teacher'][1])
img

In [ ]:
rev = inverse_normalize(tensor=out['local_crops'][1], mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
img = f.to_pil_image(rev)
img

In [ ]:
patch

In [ ]:
out.keys()

In [ ]:
%%timeit
# for i in tqdm(range(10)):
patch = Image.open(patches[0]).convert(mode="RGB")

In [ ]:
# %%timeit
# patches = list(Path("/lustre/groups/shared/histology_data/TCGA/").glob("**/*.h5"))
# patches_dx = list(Path("/lustre/groups/shared/histology_data/TCGA").glob("**/*.j[pe]g]"))
# patches_dx = list(Path("/lustre/groups/shared/histology_data/TCGA/").glob("**/patch*/0512_px_20_mag*/*DX*.h5"))
patches = np.loadtxt("/lustre/groups/shared/histology_data/TCGA_h5patches.txt", dtype=str)
# patches = np.loadtxt("/lustre/groups/shared/histology_data/TCGA/CRC/patches/512px_crc_wonorm_complete_diag_frozen.txt", dtype=str, max_rows=100).tolist()


In [ ]:
%%timeit
# open images from h5 file
# path = "/lustre/groups/shared/histology_data/TCGA/HNSC/featureses/0512_px_20_mag_otsu/TCGA-4P-AA8J-01Z-00-DX1.h5"
# for i in tqdm(range(10)):
file = h5py.File(patches[0], 'r')
patch_id = torch.randint(file['data'].shape[0], (1,)).item()
img = Image.fromarray(file['data'][patch_id])#.convert(mode="RGB")

In [ ]:
%%timeit
file = h5py.File(patches[0], 'r')
img = Image.fromarray(file['data'][0])#.convert(mode="RGB")

In [ ]:
%%timeit
# open image from h5 file
with h5py.File(patches[0], 'r') as file:
    patch_id = torch.randint(file['data'].shape[0], (1,)).item()
    img = Image.fromarray(file['data'][patch_id])#.convert(mode="RGB")

In [ ]:
# %%timeit
# open image from h5 file
file = h5py.File(patches[10], 'r')
patch_id = torch.randint(file['data'].shape[0], (1,)).item()
img = Image.fromarray(file['data'][patch_id])#.convert(mode="RGB")

In [ ]:
img

In [ ]:
file['data']

In [ ]:
len(patches)

In [ ]:
len(patches_dx)

In [ ]:
patches[0]

In [ ]:
p = []
for i in tqdm(range(1000)):
    p.append(h5py.File(patches[i], 'r'))